In [2]:
import pandas as pd

import numpy as np
import pandas as pd
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import GridSearchCV, GroupKFold, KFold
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.ensemble import RandomForestClassifier


In [3]:
data = pd.read_pickle('../create_clusters/clusters/data_clusters_t0_batch_1_4_075.pkl')

data.drop('batch', axis=1, inplace=True)

data['sample'] = data['patient_number'].astype(str) + "_" + data['test_time'].astype(str)
data

,89Y_CD45,111Cd_CD3,112Cd_CD34,113Cd_CD123,114Cd_CD66b,116Cd_HLA-DR,141Pr_CD38,142Nd_cCaspase3,143Nd_pCRKL Y207,144Nd_pTyr,...,173Yb_STAT3tot,174Yb_CD11c,175Lu_CXCR4,176Yb_pS6 S240244,195Pt_mBC2,209Bi_CD11b,patient_number,test_time,cluster,sample
0,2.217051,0.113545,3.317439,0.381279,1.590146,2.885046,3.629630,0.186837,0.327056,0.842817,...,2.558300,0.000000,0.953744,0.000000,3.512650,0.531857,01,0,17,01_0
1,3.963797,0.000000,0.311732,0.000000,4.716617,0.714334,0.808533,0.152932,0.676452,0.183566,...,2.994078,1.007841,1.161596,0.040417,3.752279,3.391960,01,0,11,01_0
2,2.961751,0.231498,0.041203,0.275047,4.226237,0.250043,0.766132,0.000000,0.075735,1.489841,...,3.148486,0.135580,0.000000,0.749384,3.559952,2.233353,01,0,11,01_0
3,3.951971,0.391544,0.030235,1.911737,1.914248,0.196068,4.125344,0.238149,0.496188,0.466622,...,2.985532,0.909775,2.843206,1.296746,3.968642,0.490613,01,0,12,01_0
4,3.204973,0.000000,0.002475,0.400668,4.130337,0.454735,0.448444,0.079543,0.328764,1.208815,...,3.489844,0.758197,0.813311,0.877981,2.142285,2.659254,01,0,11,01_0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26288919,1.693280,0.348215,0.502324,0.185988,3.035682,0.350522,2.593030,0.211198,1.259692,0.766676,...,4.092850,0.000000,0.000000,3.178231,4.793082,0.000000,HDBM_532,None,7,HDBM_532_None
26520713,0.729598,0.485712,0.172502,0.000000,0.772020,0.000000,0.719245,0.417552,1.475494,1.154216,...,3.757267,0.099147,0.294973,0.936303,4.438051,0.384407,HDBM_630,None,4,HDBM_630_None
26031470,2.254079,0.862573,0.441509,0.498153,3.449804,0.328851,1.732715,0.144498,0.869975,1.309697,...,4.134409,0.090517,0.929077,1.499913,5.032999,1.446886,HDBM_436,None,0,HDBM_436_None
27498207,4.621356,3.484828,0.077849,0.097483,0.373927,0.000000,0.195959,0.000000,0.244078,0.420155,...,1.483905,0.000000,1.437943,0.186262,3.698609,0.000000,HDPB_625,None,14,HDPB_625_None


In [4]:

internal_and_external = ['89Y_CD45', 
                '111Cd_CD3', 
                '112Cd_CD34',
                '113Cd_CD123', 
                '114Cd_CD66b', 
                '116Cd_HLA-DR', 
                '141Pr_CD38', 
                '142Nd_cCaspase3', 
                '143Nd_pCRKL Y207', 
                '144Nd_pTyr',
                '145Nd_CD4', 
                '146Nd_CD49d', 
                '147Sm_CD20', 
                '148Nd_CD16', 
                '149Sm_CD25',
                '150Nd_pSTAT5 Y694', 
                '151Eu_pSTAT3 S727', 
                '152Sm_CD13',
                '153Eu_pSTAT1 Y701', 
                '154Sm_CD45RA', 
                '155Gd_CD27',
                '156Gd_pp38 T180Y182', 
                '157Gd_CD8a', 
                '158Gd_pSTAT3 Y705',
                '159Tb_pMAPKAPK T334', 
                '160Gd_CD14', 
                '161Dy_CD26', 
                '162Dy_FoxP3',
                '163Dy_CD56', 
                '164Dy_CD15', 
                '165Ho_pCREB S133', 
                '166Er_MPO',
                '167Er_IL1-RAP', 
                '168Er_CD117', 
                '169Tm_CD33', 
                '170Er_pSRC Y418',
                '171Yb_pERK T202Y204', 
                '172Yb_pS6 S235S236', 
                '173Yb_STAT3tot',
                '174Yb_CD11c', 
                '175Lu_CXCR4', 
                '176Yb_pS6 S240244', 
                '195Pt_mBC2', 
                '209Bi_CD11b']    
        

external = ['89Y_CD45', 
                '111Cd_CD3', 
                '112Cd_CD34',
                '113Cd_CD123', 
                '114Cd_CD66b', 
                '116Cd_HLA-DR', 
                '141Pr_CD38', 
                '145Nd_CD4', 
                '146Nd_CD49d', 
                '147Sm_CD20', 
                '148Nd_CD16', 
                '149Sm_CD25',
                '152Sm_CD13',
                '154Sm_CD45RA', 
                '155Gd_CD27',
                '157Gd_CD8a', 
                '160Gd_CD14', 
                '161Dy_CD26', 
                '162Dy_FoxP3',
                '163Dy_CD56', 
                '164Dy_CD15', 
                '166Er_MPO',
                '167Er_IL1-RAP', 
                '168Er_CD117', 
                '169Tm_CD33', 
                '174Yb_CD11c', 
                '175Lu_CXCR4', 
                '195Pt_mBC2', 
                '209Bi_CD11b']

internal = list(set(internal_and_external) - set(external))

In [5]:
X = data[external]
y = data['cluster']
groups = data['sample']


In [17]:
# Initialize the classifier
model = LinearDiscriminantAnalysis()

# Hyperparameters for LDA could include different solvers and shrinkage
param_grid = [
    {'solver': ['svd'], 'shrinkage': [None]},  # SVD does not support shrinkage
    {'solver': ['lsqr', 'eigen'], 'shrinkage': [None, 'auto', 0.1, 0.5, 0.9]}
]


# Setup for K-Fold Cross-Validation
outer_cv = GroupKFold(n_splits=5)
inner_cv = GroupKFold(n_splits=3) # Inner loop for hyperparameter tuning

results = []

for train_idx, test_idx in outer_cv.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    groups_train = groups.iloc[train_idx]

    # Set up GridSearchCV
    grid_search = GridSearchCV(
        estimator=model,
        param_grid=param_grid,
        cv=inner_cv,
        scoring='accuracy'
    )

    # Fit the model
    grid_search.fit(X_train, y_train, groups=groups_train)
    best_model = grid_search.best_estimator_
    predictions = best_model.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    f1_scores = f1_score(y_test, predictions, average=None)
    median_f1 = np.median(f1_scores)

    # # Weight calculation
    # class_counts = np.bincount(y_train)
    # sample_weights = np.array([1.0 / class_counts[label] if label < len(class_counts) else 0 for label in y_test])
    # weighted_f1 = np.average(f1_scores, weights=sample_weights)

    # Additional metrics
    true_frequencies = np.bincount(y_test) / len(y_test)
    predicted_frequencies = np.bincount(predictions, minlength=len(np.unique(y))) / len(predictions)
    max_diff = np.max(np.abs(true_frequencies - predicted_frequencies))
    rsse = np.sqrt(np.sum((true_frequencies - predicted_frequencies) ** 2))

    results.append({
        'accuracy': accuracy,
        'median_f1': median_f1,
        # 'weighted_f1': weighted_f1,
        'max_diff': max_diff,
        'rsse': rsse,
        'est': grid_search.best_score_,
        'cfg': grid_search.best_params_
    })

    print('One round completed')

# Print results
results_df = pd.DataFrame(results)
print(results_df)


One round completed
One round completed
One round completed
One round completed
One round completed
   accuracy  median_f1  max_diff      rsse       est  \
0  0.857738   0.853752  0.018920  0.032343  0.868246   
1  0.859640   0.863023  0.017040  0.028528  0.869144   
2  0.880109   0.884127  0.009938  0.014800  0.865585   
3  0.863400   0.850453  0.013803  0.017486  0.861835   
4  0.887306   0.875203  0.010501  0.016446  0.863062   

                                    cfg  
0  {'shrinkage': 0.9, 'solver': 'lsqr'}  
1  {'shrinkage': 0.9, 'solver': 'lsqr'}  
2  {'shrinkage': 0.9, 'solver': 'lsqr'}  
3  {'shrinkage': 0.9, 'solver': 'lsqr'}  
4  {'shrinkage': 0.9, 'solver': 'lsqr'}  


In [18]:
model_1 = RandomForestClassifier(n_estimators=100)

# Setup for K-Fold Cross-Validation
outer_cv = GroupKFold(n_splits=5)

results_1 = []

for train_idx, test_idx in outer_cv.split(X, y, groups):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]
    groups_train = groups.iloc[train_idx]

    # Fit the model
    model_1.fit(X_train, y_train)
    predictions = model_1.predict(X_test)
    accuracy = accuracy_score(y_test, predictions)
    f1_scores = f1_score(y_test, predictions, average=None) 
    median_f1 = np.median(f1_scores)

    # Additional metrics
    true_frequencies = np.bincount(y_test) / len(y_test)
    predicted_frequencies = np.bincount(predictions, minlength=len(np.unique(y))) / len(predictions)
    max_diff = np.max(np.abs(true_frequencies - predicted_frequencies))
    rsse = np.sqrt(np.sum((true_frequencies - predicted_frequencies) ** 2))

    results_1.append({
        'accuracy': accuracy,
        'median_f1': median_f1,
        'max_diff': max_diff,
        'rsse': rsse
    })

    print('One round completed')

# Print results
results_df_rfc = pd.DataFrame(results_1)
print(results_df_rfc)

One round completed
One round completed
One round completed
One round completed
One round completed
   accuracy  median_f1  max_diff      rsse
0  0.848446   0.832118  0.009814  0.017105
1  0.850503   0.844898  0.012764  0.024750
2  0.864270   0.851744  0.015521  0.018986
3  0.859898   0.846153  0.008132  0.015112
4  0.863014   0.858457  0.011444  0.017120


## Training LDA on all cells

In [6]:
model = LinearDiscriminantAnalysis(solver='lsqr', shrinkage=0.9)

model.fit(X, y)

LinearDiscriminantAnalysis(shrinkage=0.9, solver='lsqr')

### Predicting cell clusters in pool 2

In [7]:
data_pool_2 = pd.read_pickle('../cytofdata/uncorrected_batch_5_6.pkl')

In [8]:
data_pool_2 = data_pool_2[~data_pool_2['file_id'].isin(['RT', 'RJ'])]

In [9]:
X_pool_2 = data_pool_2[external]
y_pool_2 = model.predict(X_pool_2)

data_pool_2['cluster'] = y_pool_2

In [10]:
data_pool_2['cluster'].value_counts()

cluster
1     1474253
4     1066346
2      912618
0      738782
3      466230
13     434541
11     417629
6      266657
10     240362
12     190952
8      189398
5      171043
14     159074
15     114965
9      107291
7       64551
16      60572
27      48658
21      41689
19      39883
20      37040
18      29896
23      26609
25      23231
24      15319
17      11946
26      10159
22       4256
Name: count, dtype: int64

In [11]:
data_pool_2.to_pickle('../cytofdata/data_clusters_batch_5_6_unannotated_model.pkl')

In [12]:
data_test = data[data['sample'].isin(['01_0', '01_1', '01_2',
                                   '12_0', '12_1', '12_2',
                                   '08_0', '08_1', '08_2',
                                   '15_0', '15_1', '15_2',
                                   '20_0', '20_1', '20_2',
                                   '28_0', '28_1', '28_2',
                                   '35_0', '35_1', '35_2',
                                   'HDBM_663_None', 'HDPB_597_None', 'HDBM_578_None',])]

data_train = data[~data['sample'].isin(['01_0', '01_1', '01_2',
                                   '12_0', '12_1', '12_2',
                                   '08_0', '08_1', '08_2',
                                   '15_0', '15_1', '15_2',
                                   '20_0', '20_1', '20_2',
                                   '28_0', '28_1', '28_2',
                                   '35_0', '35_1', '35_2',
                                   'HDBM_663_None', 'HDPB_597_None', 'HDBM_578_None',])]



In [13]:

X = data_train[external]
y = data_train['cluster']

model.fit(X, y)

LinearDiscriminantAnalysis(shrinkage=0.9, solver='lsqr')

In [14]:
X_test = data_test[external]
y_test = data_test['cluster']

In [15]:
data_train['cluster'].value_counts()

cluster
1     2801940
0     2422551
2     1910623
5     1480151
3     1430020
4     1032687
6      854764
8      678331
7      611163
9      587310
10     448990
11     444290
12     357445
13     323213
14     141641
19     140586
15     104153
16      80176
17      44503
21      39523
18      38636
22      36411
23      20014
25      17297
20      14493
24       8391
27       5056
26       2526
Name: count, dtype: int64

In [13]:
model.score(X_test, y_test)

0.8728267502788241

In [20]:
import plotly.express as px
from sklearn.metrics import confusion_matrix


In [21]:
y_pred = model.predict(X_test)

In [27]:
class_labels = [str(i) for i in range(28)]

## Making confusion matrix

In [29]:
cm = confusion_matrix(y_test, y_pred)

# Normalize the confusion matrix
cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

In [31]:
fig = px.imshow(
    cm_normalized,
    text_auto=True,
    labels=dict(x="Predicted Class", y="Actual Class", color="Normalized Count"),
    x=class_labels,
    y=class_labels,
    color_continuous_scale='Blues'
)

# Update layout to add titles and adjust other aspects of the layout if necessary
fig.update_layout(
    title="Normalized Confusion for the unannotated model",
    xaxis_title="Predicted Class",
    yaxis_title="Actual Class"
)

# Show the figure
fig.show()


In [35]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, y_pred)

cm_normalized = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]

fig = make_subplots(
    rows=1, cols=2,
    subplot_titles=("Original Confusion Matrix", "Normalized Confusion Matrix"),
    horizontal_spacing=0.15  
)


fig.add_trace(
    go.Heatmap(
        z=cm,
        x=class_labels,
        y=class_labels,
        colorscale='Blues',
        text=cm,
        texttemplate="%{text}",
        showscale=True
    ),
    row=1, col=1
)

fig.add_trace(
    go.Heatmap(
        z=cm_normalized,
        x=class_labels,
        y=class_labels,
        colorscale='Blues',
        text=np.round(cm_normalized, 2),
        texttemplate="%{text}",
        showscale=True
    ),
    row=1, col=2
)

fig.update_layout(
    title_text="Confusion for the unannotated model",
    xaxis_title="Predicted Class",
    yaxis_title="Actual Class",
    xaxis2_title="Predicted Class",
    yaxis2_title="Actual Class"
)

fig.show()
